In [ ]:
import pandas as pd
import plotly.express as px
import plotly.io as pio
from multicall import Call

from mainnet_launch.abis import CURVE_STABLESWAP_NG_ABI
from mainnet_launch.constants import AUTO_ETH, ETH_CHAIN
from mainnet_launch.data_fetching.alchemy.get_events import fetch_events, get_each_event_in_contract
from plotly.subplots import make_subplots
from mainnet_launch.data_fetching.get_state_by_block import (
    get_raw_state_by_blocks,
    safe_normalize_with_bool_success,
    identity_with_bool_success,
    safe_normalize_6_with_bool_success
)

pio.templates.default = "plotly_white"

POOL_ADDRESS = "0x5B03CcCAb7BA3010fA5CAd23746cbf0794938e96"  # Curve USDT-USDE pool
CSV_PATH = "./curve_usdt_usde_destination_states.csv"

df = pd.read_csv(CSV_PATH)
df["datetime"] = pd.to_datetime(df["datetime"])
df = df.set_index("datetime")
# keep only rows after December 2025 (i.e., from 2026-01-01 onward)
# df = df[df.index > "2025-10-31"].copy()

vp_call = Call(
    target=POOL_ADDRESS,
    function=["get_virtual_price()(uint256)"],
    returns=[("virtual_price", safe_normalize_with_bool_success)],
)

usdt_call = Call(
    target=POOL_ADDRESS,
    function=["balances(uint256)(uint256)", 0],
    returns=[("usdt", safe_normalize_6_with_bool_success)],
)

usde_call =Call(
    target=POOL_ADDRESS,
    function=["balances(uint256)(uint256)", 1],
    returns=[("usde", safe_normalize_with_bool_success)],
)

stored_rates_call = Call(
    target=POOL_ADDRESS,
    function=["stored_rates()(uint256[])"],
    returns=[("stored_rates", identity_with_bool_success)],
)

# Remove the individual usdt_rate and usde_rate calls
# We'll extract them from the stored_rates result after fetching

onchain_df = get_raw_state_by_blocks(
    calls=[vp_call, usdt_call, usde_call, stored_rates_call],
    blocks=df["block"].tolist(),
    chain=AUTO_ETH.chain,
)
onchain_df['percent_usdt'] = 100 * onchain_df['usdt'] / (onchain_df['usdt'] + onchain_df['usde'])
# guard against division-by-zero / NaN
onchain_df['percent_usdt'] = onchain_df['percent_usdt'].fillna(0)
onchain_df['naive_size'] = onchain_df['usdt'] + onchain_df['usde']


import plotly.graph_objects as go

fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Scatter(x=onchain_df.index, y=onchain_df['percent_usdt'], name='percent_usdt'),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=onchain_df.index, y=onchain_df['virtual_price'], name='virtual_price'),
    secondary_y=True,
)

fig.update_xaxes(title_text="datetime")
fig.update_yaxes(title_text="percent_usdt", secondary_y=False)
fig.update_yaxes(title_text="virtual_price", secondary_y=True)
fig.update_layout(title_text="USDT Percentage and Virtual Price")

fig.show()
onchain_df.head()


In [ ]:
onchain_df.head()


In [92]:
# Merge onchain_df and df on block
merged_df = onchain_df.merge(df[['block', 'lp_token_spot_price', 'lp_token_safe_price']], 
                               left_on=onchain_df.index, 
                               right_on=df.index, 
                               how='left')
merged_df = merged_df.set_index('key_0')

# Create the plot with multiple traces on secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Scatter(x=merged_df.index, y=merged_df['percent_usdt'], name='percent_usdt'),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=merged_df.index, y=merged_df['virtual_price'], name='virtual_price'),
    secondary_y=True,
)

fig.add_trace(
    go.Scatter(x=merged_df.index, y=merged_df['lp_token_spot_price'], name='lp_token_spot_price'),
    secondary_y=True,
)

fig.add_trace(
    go.Scatter(x=merged_df.index, y=merged_df['lp_token_safe_price'], name='lp_token_safe_price'),
    secondary_y=True,
)

fig.update_xaxes(title_text="datetime")
fig.update_yaxes(title_text="percent_usdt", secondary_y=False)
fig.update_yaxes(title_text="price", secondary_y=True)
fig.update_layout(title_text="USDT Percentage and Prices")

fig.show()

In [93]:
onchain_df

,virtual_price,usdt,usde,usdt_rate,usde_rate,percent_usdt,naive_size
timestamp,,,,,,,
2025-04-03 19:27:59+00:00,1.006339,9.342459e+05,3.913617e+06,None,3.913617e+06,19.271292,4.847863e+06
2025-04-03 23:44:11+00:00,1.006348,9.734301e+05,4.145901e+06,None,4.145901e+06,19.014791,5.119331e+06
2025-04-04 15:55:11+00:00,1.006466,1.033097e+06,4.140798e+06,None,4.140798e+06,19.967493,5.173896e+06
2025-04-04 17:08:35+00:00,1.006473,7.995051e+05,4.374517e+06,None,4.374517e+06,15.452294,5.174022e+06
2025-04-04 18:56:23+00:00,1.006475,8.246081e+05,4.349411e+06,None,4.349411e+06,15.937476,5.174019e+06
...,...,...,...,...,...,...,...
2026-01-12 07:53:47+00:00,1.020793,4.662560e+05,8.430693e+04,None,8.430693e+04,84.687141,5.505630e+05
2026-01-12 08:15:47+00:00,1.020793,4.662240e+05,8.433893e+04,None,8.433893e+04,84.681328,5.505629e+05
2026-01-12 13:53:59+00:00,1.020795,4.683622e+05,8.220257e+04,None,8.220257e+04,85.069409,5.505647e+05


In [89]:
px.line(onchain_df['naive_size'])